In [1]:
#API to get airflight status
#r1 = requests.get(url='https://aviation-edge.com/v2/public/flights?key=a24d93-2501aa&arrIata=LIM')

In [2]:
import requests
import json
import numpy
import Clases as SA
from datetime import datetime, timedelta
import random
from scipy.stats import f_oneway
from scipy import stats

In [3]:
class Individual_flightAssig:
    "Clase que implementa el individuo en el problema de las n-reinas."

    def __init__(self, chromosome):
        self.chromosome = chromosome[:]
        self.fitness = -1

    def crossover_onepoint(self, other):
        "Retorna dos nuevos individuos del cruzamiento de un punto entre self y other"
        c = random.randrange(len(self.chromosome))
        ind1 = Individual_flightAssig(self.chromosome[:c] + other.chromosome[c:])
        ind2 = Individual_flightAssig(other.chromosome[:c] + self.chromosome[c:])
        return [ind1, ind2]
    
    def crossover_permutation(self, other):
        "Retorna dos nuevos individuos del cruzamiento de permutacion entre self y other" 
        "Toma una subsecuencia del cromosoma self (de tamaño igual a la mitad del cromosoma) y lo copia a un hijo ind1 " 
        "en las mismas posiciones. Los numeros faltantes los copia de other en el mismo orden en que aparecen en other "
        "El hijo ind2 se crea de la misma forma intercambiando self por other"
        #TODO
        
        
        n = len(self.chromosome)
        
        hijo1 = [0] * n
        hijo2 = [0] * n
        
        mitad = round(n/2)
        
        isubsec = random.randint(0, mitad)
        i_other, i_self  = 0, 0

        
        #copiar las mitades  en los hijos
        hijo1[isubsec:isubsec+mitad] = self.chromosome[isubsec:isubsec+mitad]
        hijo2[isubsec:isubsec+mitad] = other.chromosome[isubsec:isubsec+mitad]       

        
    
        for i in range(n):
            #en el hijo, buscamos los espacios en blanco
            if hijo1[i] is 0:
                #encontramos el siguiente numero no presente en el hijo
                while other.chromosome[i_other] in hijo1:
                    i_other+= 1          
                #lo copiamos en el hijo, en la posicion en blanco
                hijo1[i] = other.chromosome[i_other]
                
        #de la misma forma para el otro hijo
        for i in range(n):
            if hijo2[i] is 0:
                while self.chromosome[i_self] in hijo2:
                    i_self+= 1                
                hijo2[i] = self.chromosome[i_self]
        
        
                 
        ind1 = Individual_flightAssig(hijo1)
        ind2 = Individual_flightAssig(hijo2)
        
        
        return [ind1, ind2]
        
    
    def crossover_uniform(self, other):
        chromosome1 = []
        chromosome2 = []
        "Retorna dos nuevos individuos del cruzamiento uniforme entre self y other "
        for i in range(len(self.chromosome)):
            if random.uniform(0, 1) < 0.5:
                chromosome1.append(self.chromosome[i])
                chromosome2.append(other.chromosome[i])
            else:
                chromosome1.append(other.chromosome[i])
                chromosome2.append(self.chromosome[i])
        ind1 = Individual_flightAssig(chromosome1)
        ind2 = Individual_flightAssig(chromosome2)
        return [ind1, ind2] 

    def mutate_position(self):
        "Cambia aleatoriamente la posicion de una reina."
        mutated_ind = Individual_flightAssig(self.chromosome[:])
        indexPos = random.randint(0, len(mutated_ind.chromosome)-1)
        mutated_ind.chromosome[indexPos].esPuerta = random.choice([True, False])
        if(mutated_ind.chromosome[indexPos].esPuerta):
            mutated_ind.chromosome[indexPos].area = random.randint(0,numPuertas-1)
        else:
            mutated_ind.chromosome[indexPos].area = random.randint(0,numZonas-1)
        #newPos = random.randint(0, len(mutated_ind.chromosome)-1)
        #mutated_ind.chromosome[indexPos] = newPos
        return mutated_ind
        
        
    def mutate_swap(self):
        "Intercambia la posicion de dos genes."
        mutated_ind = Individual_flightAssig(self.chromosome[:])
        indexOne = random.randint(0,len(mutated_ind.chromosome)-1)
        indexTwo = random.randint(0,len(mutated_ind.chromosome)-1)
        temp = mutated_ind.chromosome[indexOne]
        mutated_ind.chromosome[indexOne].esPuerta = mutated_ind.chromosome[indexTwo].esPuerta
        mutated_ind.chromosome[indexOne].area = mutated_ind.chromosome[indexTwo].area
        mutated_ind.chromosome[indexTwo].esPuerta = temp.esPuerta
        mutated_ind.chromosome[indexTwo].area = temp.area
        return mutated_ind

In [8]:
def select_survivors(population, offspring_population, numsurvivors):
    next_population = []
    population.extend(offspring_population) # une las dos poblaciones
    isurvivors = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:numsurvivors]
    for i in range(numsurvivors): next_population.append(population[isurvivors[i]])
    return next_population

In [9]:
def select_parents_roulette(population):
    popsize = len(population)
    
    # Escoje el primer padre
    sumfitness = sum([indiv.fitness for indiv in population])  # suma total del fitness de la poblacion
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent1 = i
            break
     
    # Escoje el segundo padre, desconsiderando el primer padre
    sumfitness = sumfitness - population[iParent1].fitness # retira el fitness del padre ya escogido
    pickfitness = random.uniform(0, sumfitness)   # escoge un numero aleatorio entre 0 y sumfitness
    cumfitness = 0     # fitness acumulado
    for i in range(popsize):
        if i == iParent1: continue   # si es el primer padre 
        cumfitness += population[i].fitness
        if cumfitness > pickfitness: 
            iParent2 = i
            break        
    return (population[iParent1], population[iParent2])

In [10]:
def evaluate_population(population, fitness_fn, numPuertas, numZonas):
    """ Evalua una poblacion de individuos con la funcion de fitness pasada """
    popsize = len(population)
    for i in range(popsize):
        if population[i].fitness == -1:    # si el individuo no esta evaluado
            population[i].fitness = fitness_fn(population[i].chromosome, numPuertas, numZonas)

In [4]:
class ColaDeVuelos:
    def __init__ (self):
        self.tiempoSinUsar = 0
        self.tiempoHorario = None        


In [148]:
def calcularTiempoSinUso(colaDeVuelo, vuelo, parCastigo):
    colaDeVuelo.tiempoSinUsar = colaDeVuelo.tiempoSinUsar + parCastigo*((vuelo.tiempoProgramado-colaDeVuelo.tiempoHorario).seconds)**2
    colaDeVuelo.tiempoHorario = vuelo.tiempoProgramado +  timedelta(hours=3)
    return

In [149]:
def calcularTiempoDeEspera(colaDeVuelo, vuelo):
    auxTiempoEspera = 0
    auxTiempoEspera = ((colaDeVuelo.tiempoHorario-vuelo.tiempoProgramado).seconds)**2
    colaDeVuelo.tiempoHorario = colaDeVuelo.tiempoHorario + timedelta(hours=3)
    return auxTiempoEspera

In [129]:
 def fitness_gates(chromosome, numPuertas, numZonas):
    """Retorna el fitness de un cromosoma en el problema de vuelos (nro de pares de reinas no atacadas) """
    n = len(chromosome)  # nro de vuelos
    fitness = 0
    listPuertas = [] 
    listZonas = []
    for i in range(numPuertas):
        listPuertas.append(ColaDeVuelos())
        
    for i in range(numZonas):
        listZonas.append(ColaDeVuelos())
        
    for i in range(n):
        auxTiempoEspera = 0
        auxArea = chromosome[i].area
        if(chromosome[i].esPuerta==True):
            parCastigo = 900000
            if (listPuertas[auxArea].tiempoHorario==None):
                listPuertas[auxArea].tiempoHorario = chromosome[i].tiempoProgramado
            elif (listPuertas[auxArea].tiempoHorario < chromosome[i].tiempoProgramado):
                calcularTiempoSinUso(listPuertas[auxArea],chromosome[i],parCastigo)
            elif (listPuertas[auxArea].tiempoHorario >= chromosome[i].tiempoProgramado):
                auxTiempoEspera = auxTiempoEspera + calcularTiempoDeEspera(listPuertas[auxArea],chromosome[i])
        else:
            parCastigo = 1
            if (listZonas[auxArea].tiempoHorario==None):
                listZonas[auxArea].tiempoHorario = chromosome[i].tiempoProgramado
            elif (listZonas[auxArea].tiempoHorario < chromosome[i].tiempoProgramado):
                calcularTiempoSinUso(listZonas[auxArea],chromosome[i],parCastigo)
            elif (listZonas[auxArea].tiempoHorario >= chromosome[i].tiempoProgramado):
                auxTiempoEspera = auxTiempoEspera + calcularTiempoDeEspera(listZonas[auxArea],chromosome[i])
            
    auxTiempoSinUso = 0
    for i in range(numPuertas-1):
        auxTiempoSinUso = auxTiempoSinUso + listZonas[i].tiempoSinUsar
        
    for i in range(numZonas-1):
        auxTiempoSinUso = auxTiempoSinUso + listZonas[i].tiempoSinUsar
        
    fitness = fitness + auxTiempoEspera + auxTiempoSinUso
    fitness = 1/fitness
    return fitness

In [115]:
def genetic_algorithm(population, fitness_fn, numPuertas, numZonas, ngen=100, pmut=0.1, crossover="onepoint", mutation="position"):
    "Algoritmo Genetico "

    popsize = len(population)
    evaluate_population(population, fitness_fn, numPuertas, numZonas)  # evalua la poblacion inicial
    ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
    bestfitness = [population[ibest[0]].fitness]
    
    for g in range(ngen):   # Por cada generacion
        
        ## Selecciona las parejas de padres para cruzamiento 
        mating_pool = []
        for i in range(int(popsize/2)): mating_pool.append(select_parents_roulette(population)) 
        
        ## Crea la poblacion descendencia cruzando las parejas del mating pool 
        offspring_population = []
        for i in range(len(mating_pool)): 
            if crossover == "onepoint":
                offspring_population.extend( mating_pool[i][0].crossover_onepoint(mating_pool[i][1]) ) # cruzamiento 1 punto
            elif crossover == "uniform":
                offspring_population.extend( mating_pool[i][0].crossover_uniform(mating_pool[i][1]) ) # cruzamiento uniforme
            elif crossover == "permutation":    
                offspring_population.extend( mating_pool[i][0].crossover_permutation(mating_pool[i][1]) ) # cruzamiento de permutacion

        ## Aplica el operador de mutacion con probabilidad pmut en cada hijo generado
        for i in range(len(offspring_population)):
            if random.uniform(0, 1) < pmut: 
                if mutation == "position":
                    offspring_population[i] = offspring_population[i].mutate_position()   # mutacion de una posicion
                elif mutation == "swap":
                    offspring_population[i] = offspring_population[i].mutate_swap()      # mutacion swap
        
        ## Evalua la poblacion descendencia creada
        evaluate_population(offspring_population, fitness_fn, numPuertas, numZonas)  # evalua la poblacion inicial
        
        ## Selecciona popsize individuos para la sgte. generación de la union de la pob. actual y  pob. descendencia
        population = select_survivors(population, offspring_population, popsize)

        ## Almacena la historia del fitness del mejor individuo
        ibest = sorted(range(len(population)), key=lambda i: population[i].fitness, reverse=True)[:1]
        #bestfitness.append(population[ibest[0]].fitness)
        #imprimirAsignacion(population[ibest[0]])
        #print("--------------------------------------------------------------------------")
        #print("generacion {}, best_fitness = {}".format(g, population[ibest[0]].fitness))
    
    return population[ibest[0]], bestfitness  

In [109]:
def genetic_AGAP(fitness_fn, listaVuelos, numPuertas, numZonas, popsize=8, ngen=100, pmut=0.1, crossover="onepoint", mutation="position"):
    import random
    population = []
                   
    for i in range(popsize):
        listaAssig=listaVuelos[:]
        Individual_flightAssig(listaAssig)
        for j in range(len(listaAssig)):
            listaAssig[j].esPuerta = random.choice([True, False])
            if(listaAssig[j].esPuerta):
                listaAssig[j].area = random.randint(0,numPuertas-1)
            else:
                listaAssig[j].area = random.randint(0,numZonas-1)
        
        population.append(Individual_flightAssig(listaAssig))
        
        
        
    ## llama al algoritmo genetico para encontrar una solucion al problema de las n reinas
    return genetic_algorithm(population, fitness_fn, numPuertas, numZonas, ngen, pmut, crossover, mutation)        

In [3]:
#API to get schedules
r2 = requests.get(url='https://aviation-edge.com/v2/public/timetable?key=a24d93-2501aa&iataCode=LIM&type=arrival')

In [4]:
data = r2.json()
print (data)

[{'type': 'arrival', 'status': 'landed', 'departure': {'iataCode': 'LPB', 'icaoCode': 'SLLP', 'gate': 'R19', 'scheduledTime': '2019-05-20T07:56:00.000', 'estimatedTime': '2019-05-20T07:52:00.000', 'actualTime': '2019-05-20T07:52:00.000', 'estimatedRunway': '2019-05-20T08:06:00.000', 'actualRunway': '2019-05-20T08:06:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'baggage': '4', 'scheduledTime': '2019-05-20T09:05:00.000', 'estimatedTime': '2019-05-20T08:59:00.000', 'actualTime': '2019-05-20T08:59:00.000', 'estimatedRunway': '2019-05-20T08:46:00.000', 'actualRunway': '2019-05-20T08:46:00.000'}, 'airline': {'name': 'AVIANCA', 'iataCode': 'AV', 'icaoCode': 'AVA'}, 'flight': {'number': '7391', 'iataNumber': 'AV7391', 'icaoNumber': 'AVA7391'}, 'codeshared': {'airline': {'name': 'Avianca Ecuador', 'iataCode': '2K', 'icaoCode': 'GLG'}, 'flight': {'number': '7391', 'iataNumber': '2K7391', 'icaoNumber': 'GLG7391'}}}, {'type': 'arrival', 'status': 'landed', 'departure': {'iataCode':

In [98]:
def obtenerListaVuelos(data_filtered):
    listaVuelos = []
    for flight in data_filtered:
        auxVuelo = SA.Vuelo()
        if('estimatedTime' in flight['arrival']):
            auxVuelo.addEstado(flight['status'])
            auxVuelo.addAvion(flight['flight']['iataNumber'])
            auxVuelo.addTiempoEstimado(datetime.strptime(flight['arrival']['estimatedTime'][:-4], '%Y-%m-%dT%H:%M:%S'))
            auxVuelo.addTiempoProgramado(datetime.strptime(flight['arrival']['scheduledTime'][:-4], '%Y-%m-%dT%H:%M:%S'))
            listaVuelos.append(auxVuelo)
        else:
            auxVuelo.addEstado(flight['status'])
            auxVuelo.addAvion(flight['flight']['iataNumber'])
            auxVuelo.addTiempoProgramado(datetime.strptime(flight['arrival']['scheduledTime'][:-4], '%Y-%m-%dT%H:%M:%S'))
            listaVuelos.append(auxVuelo)
    return listaVuelos

In [99]:
def obtenerData(nombreArchivo):
    with open(nombreArchivo) as json_file:  
        data = json.loads(json_file.read().replace("\'", "\""))
        data_filtered = list(filter(lambda x : x['status'] != 'landed' and x['status'] != 'cancelled', data))
    return data_filtered

In [100]:
def imprimirAsignacion(asignacion):
    n = len(asignacion.chromosome)
    listPuertas = [] 
    listZonas = []
    for i in range(numPuertas):
        listPuertas.append(ColaDeVuelos())
        
    for i in range(numZonas):
        listZonas.append(ColaDeVuelos())
        
    for i in range(n):
        auxArea = asignacion.chromosome[i].area
        if(asignacion.chromosome[i].esPuerta==True):
            if (listPuertas[auxArea].tiempoHorario == None):
                listPuertas[auxArea].tiempoHorario = asignacion.chromosome[i].tiempoProgramado
                print("P",auxArea,"\t",listPuertas[auxArea].tiempoHorario)
            elif (listPuertas[auxArea].tiempoHorario < asignacion.chromosome[i].tiempoProgramado):
                listPuertas[auxArea].tiempoHorario = asignacion.chromosome[i].tiempoProgramado +  timedelta(hours=2)
                print("P",auxArea,"\t",listPuertas[auxArea].tiempoHorario)
            elif (listPuertas[auxArea].tiempoHorario >= asignacion.chromosome[i].tiempoProgramado):
                listPuertas[auxArea].tiempoHorario = listPuertas[auxArea].tiempoHorario + timedelta(hours=2)
                print("P",auxArea,"\t",listPuertas[auxArea].tiempoHorario)
        else:
            if (listZonas[auxArea].tiempoHorario==None):
                listZonas[auxArea].tiempoHorario = asignacion.chromosome[i].tiempoProgramado
                print("Z",auxArea,"\t",listZonas[auxArea].tiempoHorario)
            elif (listZonas[auxArea].tiempoHorario < asignacion.chromosome[i].tiempoProgramado):
                listZonas[auxArea].tiempoHorario = asignacion.chromosome[i].tiempoProgramado +  timedelta(hours=2)
                print("Z",auxArea,"\t",listZonas[auxArea].tiempoHorario)
            elif (listZonas[auxArea].tiempoHorario >= asignacion.chromosome[i].tiempoProgramado):
                listZonas[auxArea].tiempoHorario = listZonas[auxArea].tiempoHorario + timedelta(hours=2)
                print("Z",auxArea,"\t",listZonas[auxArea].tiempoHorario)

In [178]:
data_filtered = obtenerData("ArrivalLima190509.txt")
listaVuelos = obtenerListaVuelos(data_filtered)
numVuelos = len(data_filtered)
numPuertas = 20
numZonas = 52
popsize = 100
ngen=450
pmut = 0.2
crossover = "permutation"
mutation = "position"
asignacion,listaFitness=genetic_AGAP(fitness_gates, listaVuelos, numPuertas, numZonas, popsize, ngen, pmut, crossover, mutation)
imprimirAsignacion(asignacion)

P 0 	 2019-05-08 13:00:00
P 19 	 2019-05-08 13:00:00
Z 3 	 2019-05-08 13:00:00
P 7 	 2019-05-08 13:00:00
Z 36 	 2019-05-08 13:00:00
P 0 	 2019-05-08 15:03:00
P 2 	 2019-05-08 13:03:00
Z 27 	 2019-05-08 13:20:00
Z 2 	 2019-05-08 13:20:00
Z 17 	 2019-05-08 13:29:00
P 14 	 2019-05-08 13:29:00
P 12 	 2019-05-08 13:29:00
P 8 	 2019-05-08 13:32:00
Z 20 	 2019-05-08 13:32:00
P 5 	 2019-05-08 13:42:00
P 6 	 2019-05-08 13:42:00
Z 45 	 2019-05-08 13:42:00
P 16 	 2019-05-08 13:44:00
Z 10 	 2019-05-08 13:44:00
P 3 	 2019-05-08 13:53:00
Z 14 	 2019-05-08 14:02:00
P 19 	 2019-05-08 16:02:00
P 12 	 2019-05-08 16:02:00
P 6 	 2019-05-08 16:02:00
Z 51 	 2019-05-08 14:02:00
P 14 	 2019-05-08 15:51:00
Z 21 	 2019-05-08 13:51:00
Z 26 	 2019-05-08 13:51:00
P 18 	 2019-05-08 14:10:00
P 16 	 2019-05-08 16:10:00
P 10 	 2019-05-08 14:15:00
P 0 	 2019-05-08 17:03:00
P 8 	 2019-05-08 16:17:00
P 16 	 2019-05-08 18:10:00
P 17 	 2019-05-08 14:34:00
Z 11 	 2019-05-08 14:34:00
Z 18 	 2019-05-08 14:35:00
P 18 	 2019-05

In [179]:
print(listaFitness[-1])

1.1878698343639054e-10


In [180]:
print (1/listaFitness[-1])

8418430800.0


In [189]:
data1, data2 =  [2341605600.0,5805414000.0,5710899600.0,2806239600.0,17850722400.0,8418430800.0] , [11913099623980.0,3673568998780.0,3775397965180.0,4118026575580.0,8,3744698889580.0]
stat, p = f_oneway(data1, data2)

In [190]:
print(p)

0.02451335695149537


In [186]:
stats.describe(data1)

DescribeResult(nobs=6, minmax=(2341605600.0, 17850722400.0), mean=7155552000.0, variance=3.239640484385241e+19, skewness=1.219160719007268, kurtosis=0.19901922557924445)